In [3]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import os

scraping_path = os.path.abspath(r'C:\Users\carlf\Documents\GitHub\lyrics_generator\scraping')
sys.path.append(scraping_path)

# Importer la fonction du fichier scrapping_wiki.py
from scrapping_wiki import clean_texts

x = clean_texts()

In [71]:
def remove_elements_before(lst, target):
    if target in lst:
        index = lst.index(target)  # Trouve l'index de l'élément cible
        return lst[index:]         # Garde uniquement les éléments à partir de cet index
    else:
        return lst                 # Si la cible n'est pas dans la liste, retourne la liste d'origine

z = remove_elements_before(x, 'Youv Dee')
print(z)

['Youv Dee', 'Jean-Pascal Zadi', 'Zed (rappeur)', 'Sofiane Zermani', 'Darryl Zeuja', 'Zuukou Mayzie']


In [12]:
z = ["Alonzo", "Axiom","Jacky__BenJ","Eppsito","9Jared","Black_Kent","Daddy_Lord_C","DJ_Dee_Nasty","Favé_FRA", "Maes", "Anthony_Kazadi", "Gato_da_Bato","Akhenaton__Shurikn", "Zola_Nzuzi", "Sopra_Mbaba"]
new_artists = ["Alonzo", "Axiom", "Neg'Marrons", "Benjamin Epps", "Big Red", "Black Kent", "Daddy Lord C", "Dee Nasty", "Favé","Maes","Kaza","Gato", "IAM","Zola", "Soprano" ]

print(len(z), len(new_artists))

14 14


In [27]:
import pandas as pd 
for artist in z:
    file_name = fr'data\01_raw\{artist}_songs.csv'  # Nom du fichier CSV (format : artist_songs.csv)
    df = pd.read_csv(file_name)
    if "GeniusName.1" in df.columns:
        df = df.drop(columns="GeniusName.1")
    if "GeniusName.2" in df.columns:
        df = df.drop(columns="GeniusName.2")
    
    if "GeniusName.3" in df.columns:
        df = df.drop(columns="GeniusName.3")
        
    df.to_csv(f'data/01_raw/{artist}_songs.csv', index=False)

    

In [4]:
import pandas as pd
for artist, new_artist in zip(z, new_artists):
    # Charger le DataFrame à partir du fichier CSV
    file_name = fr'data\01_raw\{artist}_songs.csv'  # Nom du fichier CSV (format : artist_songs.csv)
    df = pd.read_csv(file_name)
    
    # Modifier le nom de la colonne 'Artist' en 'GeniusName'
    df = df.rename(columns={'Artist': 'GeniusName'})
    
    # Ajouter la colonne avec le nouveau nom de l'artiste
    df['Artist'] = new_artist
    
    # Sauvegarder les modifications dans un nouveau fichier CSV ou écraser l'existant
    df.to_csv(file_name, index=False)

In [5]:
z= ["Rohff"]

## Second method we used (no scrape just apis call)

In [6]:
import lyricsgenius
import pandas as pd
import os
from tqdm import tqdm

# Initialisation avec votre clé d'API Genius
GENIUS_API_TOKEN = "vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG"
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# Fonction pour récupérer les paroles et les métadonnées d'un artiste
def get_lyrics_with_metadata(artist_name, max_songs=500):
    print(f"Récupération des données pour : {artist_name}")
    artist = genius.search_artist(artist_name, max_songs=max_songs, sort="title")
    if artist is None:
        print(f"Aucun résultat trouvé pour {artist_name}")
        return []

    songs_data = []
    for song in tqdm(artist.songs):
        songs_data.append({
            "Artist": artist_name,
            "Title": song.title,
            "Lyrics": song.lyrics
        })

    return songs_data

# Sauvegarder les données pour un artiste dans un fichier CSV
def save_artist_lyrics_to_csv(artist_name, data, folder_path):
    # Créer le dossier s'il n'existe pas
    os.makedirs(folder_path, exist_ok=True)
    
    # Construire le chemin du fichier CSV
    file_path = os.path.join(folder_path, f"{artist_name.replace(' ', '_')}_songs.csv")
    
    # Sauvegarder les données dans un DataFrame
    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False, encoding="utf-8")
    
    print(f"Fichier CSV sauvegardé pour {artist_name} à : {file_path}")

# Exemple d'utilisation
if __name__ == "__main__":
    # Liste des rappeurs
    max_chansons = 800

    # Chemin du dossier où sauvegarder les fichiers CSV
    folder_path = r"C:\Users\carlf\Documents\GitHub\lyrics_generator\data\01_raw"

    for artist in z:
        # Récupérer les paroles et les métadonnées de l'artiste
        artist_data = get_lyrics_with_metadata(artist, max_songs=max_chansons)
        
        # Sauvegarder dans un fichier CSV spécifique à l'artiste
        save_artist_lyrics_to_csv(artist, artist_data, folder_path)


Récupération des données pour : Rohff
Searching for songs by Rohff...



HTTPError: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=Rohff

In [9]:
import requests

def get_song_release_year(genius_api_key, song_title, artist_name):
    base_url = "https://api.genius.com"
    headers = {"Authorization": f"Bearer {genius_api_key}"}
    
    # Étape 1 : Rechercher la chanson
    search_url = f"{base_url}/search"
    params = {"q": f"{song_title} {artist_name}"}
    response = requests.get(search_url, headers=headers, params=params)
    search_results = response.json()
    
    # Vérification des résultats
    if search_results.get("meta", {}).get("status") != 200:
        print("Erreur lors de la recherche.")
        return None
    
    hits = search_results.get("response", {}).get("hits", [])
    if not hits:
        print(f"Aucun résultat trouvé pour {song_title} de {artist_name}.")
        return None
    
    # On prend la première chanson trouvée
    song_id = hits[0]["result"]["id"]
    
    # Étape 2 : Récupérer les détails de la chanson
    song_url = f"{base_url}/songs/{song_id}"
    song_response = requests.get(song_url, headers=headers)
    song_data = song_response.json()
    
    # Vérifier les données de la chanson
    if song_data.get("meta", {}).get("status") != 200:
        print(f"Erreur lors de la récupération des détails de la chanson {song_title}.")
        return None
    
    song_info = song_data.get("response", {}).get("song", {})
    
    # Extraire la date de sortie
    release_date = song_info.get("release_date")
    if release_date:
        return release_date.split("-")[0]  # Extraire uniquement l'année
    else:
        print(f"Aucune date de sortie trouvée pour {song_title}.")
        return "Inconnu"

# Exemple d'utilisation
genius_api_key = "vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG"
song_title = "FREESTYLE HORS SÉRIE"
artist_name = "1PLIKÉ140"

year = get_song_release_year(genius_api_key, song_title, artist_name)
print(f"L'année de sortie de '{song_title}' est : {year}")


L'année de sortie de 'FREESTYLE HORS SÉRIE' est : 2020


## First function we used with scraping

In [1]:
import lyricsgenius as lg
import requests
import pandas as pd 
import random 
import time
import os
from tqdm import tqdm


GENIUS_API_TOKEN = "vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG"
genius = lg.Genius(GENIUS_API_TOKEN)


def _extracted_from_get_artist_id_(search_url, artist_name):
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    response = requests.get(search_url, headers=headers, timeout=120)
    response.raise_for_status()  # Vérifie les erreurs HTTP
    data = response.json()

    if 'response' in data and data['response']['hits']:
        return data['response']['hits'][0]['result']['primary_artist']['id']
    print(f"Aucune donnée trouvée pour l'artiste: {artist_name}")
    return None

def get_artist_id(artist_name):
    """Récupère l'ID d'un artiste en utilisant l'API Genius."""
    search_url = f"https://api.genius.com/search?q={artist_name}"
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}

    try:
        return _extracted_from_get_artist_id_(search_url, artist_name)
    except Exception as e:
        print(f"Erreur lors de la récupération de l'ID pour {artist_name}: {e}")
        return None

def get_songs(artist_id, max_songs=500):
    """Récupère jusqu'à un nombre donné de chansons pour un artiste, triées par popularité."""
    songs = []
    page = 1
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs?page={page}"
        headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
        response = requests.get(url, headers=headers).json()
        songs_data = response['response']['songs']
        
        if not songs_data:
            break  # Sortir si aucune chanson n'est trouvée

        # Récupérer la popularité et ajouter les chansons à la liste
        for song in songs_data:
            if len(songs) < max_songs:  # Assurez-vous de ne pas dépasser max_songs
                songs.append({
                    'title': song['title'],
                    'popularity': song.get('stats', {}).get('pageviews', 0)  # Exemple de mesure de popularité
                })
            else:
                break  # Sortir de la boucle for si le maximum est atteint

        page += 1  # Passer à la page suivante
        time.sleep(random.uniform(1,3))
        
    # Trier manuellement par popularité
    sorted_songs = sorted(songs, key=lambda x: x['popularity'], reverse=True)
    return [song['title'] for song in sorted_songs[:max_songs]]  # Limiter le retour à max_songs

def download_lyrics(song_title, artist_name):
    """Télécharge les paroles d'une chanson donnée."""

    if song := genius.search_song(song_title, artist_name):
        try :
            paroles = song.lyrics
        except Exception as e :
            print(f"Erreur lors de la récupération des paroles : {e}")
            return None

        return paroles, artist_name, song_title
    return None

def all_artist_songs(artist_to_scrape):
    """Télécharge les paroles de toutes les chansons pour chaque artiste et les stocke dans un fichier CSV par artiste."""
    for artist in artist_to_scrape:
        artist_id = get_artist_id(artist)
        if artist_id is None:
            print(f"ID d'artiste non trouvé pour {artist}")

        elif songs := get_songs(artist_id):
            # Créer un répertoire pour l'artiste si il n'existe pas
            os.makedirs(r"C:/Users/carlf/Documents/GitHub/lyrics_generator/data/01_raw", exist_ok=True)
            artist_lyrics = [] 
            for song in tqdm(songs):
                if lyrics_info := download_lyrics(song, artist):
                    lyrics, artist, song = lyrics_info
                    artist_lyrics.append({
                        'Artist': artist,
                        'Title': song,
                        'Lyrics': lyrics
                    }) 
                    # print(f"Téléchargé : {song} de {artist}")
                    sleep_time = random.uniform(0.3,0.65)
                    # print(f"Sleeping for {sleep_time:.2f} seconds before the next download...")
                    time.sleep(sleep_time)
                else:
                    print(f"Paroles non trouvées pour {song} de {artist}")

            df = pd.DataFrame(artist_lyrics)
            # Enregistrer le DataFrame dans un fichier CSV
            df.to_csv(rf"C:/Users/carlf/Documents/GitHub/lyrics_generator/data/01_raw/{artist}_songs.csv", index=False, encoding='utf-8')
            print(f"Fichier créé : {artist}_songs.csv")
        else:
            print(f"Aucune chanson trouvée pour {artist}")



In [21]:
z = ["Fonky Family","IAM", "Sexion d'assaut", "Medine", "Ziak","Chily", "113 clan","Anthony Kazadi" "Casseurs Flowters", "El Matador", "Zola", "Gato", "Soprano", "Akhenaton", "Nepal", "Maes" "Leto", "Josman", "Naps","Lino","Disiz", "Ninho","Alkpote","Orelsan", "Gazo", "Kerchak", "Fave", "Jul", "Larry", "Siboy", "MHD", "Soolking","Kekra", "Kaaris","Medine","Lomepal","SCH", "Caballero & JeanJass", "Laylow","Youssoupha", "Vald", "Dinos","Fianso", "Dosseh", "Diam's","Lefa","Despo Rutti", "Hamza","Krisy","Tusiano","Casey","Kalash","Roméo Elvis","Rohff","PLK", "Menace Santana","13 Block", "O'boy","Sneazzy", "DTF","Sadek","Lesram", "Kodes","Diddy Trix","Gambi", "LuvResval", "Werenoi", "Green Santana","M.I.G", "Zequin", "SDM","Youv Dee", "Ash Kidd", "Lyonzon", ]
# e = 0
# for i in z :
#     e +=1
# print(e)

In [20]:
z = ["Gazo", "Kerchak", "Fave", "Jul", "Larry", "Siboy", "MHD", "Soolking","Kekra", "Kaaris","Medine","Lomepal","SCH", "Caballero & JeanJass", "Laylow","Youssoupha", "Vald", "Dinos","Fianso", "Dosseh", "Diam's","Lefa","Despo Rutti", "Hamza","Krisy","Tusiano","Casey","Kalash","Roméo Elvis","Rohff","PLK", "Menace Santana","13 Block", "O'boy","Sneazzy", "DTF","Sadek","Lesram", "Kodes","Diddy Trix","Gambi", "LuvResval", "Werenoi", "Green Santana","M.I.G", "Zequin", "SDM","Youv Dee", "Ash Kidd", "Lyonzon", ]

In [2]:
z = ["Elio Menais", "Rohff"]

In [4]:
all_artist_songs(z)

  0%|          | 0/99 [00:00<?, ?it/s]

Searching for "24 Hour" by Elio Menais...


  0%|          | 0/99 [00:03<?, ?it/s]


HTTPError: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=24+Hour+Elio+Menais